In [44]:
import acquire

import pandas as pd

In [138]:
df = acquire.get_brazil_data()
df.head()

/usr/local/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (3,5,6,7,8,9,10,13,14,23,24,25,27,28,29,30,31,32,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,Patient.Code,RCBP.Name,Gender,Date.of.Birth,Age,Raca.Color,Nationality,Naturality.State,Naturality,Degree.of.Education,...,Laterality,Statement,TNM,Status.Vital,Type.of.Death,Date.of.Death,Date.of.Last.Contact,Date.of.Diagnostic,Distant.metastasis,year
0,672054.0,RCBP CAMPINAS-UNICAMP,MASCULINO,1975-03-16,30.0,BRANCO,BRASIL,S�O PAULO,NaN,NaN,...,NaN,NaN,NaN,MORTO,C�NCER,2005-10-07,2005-10-07,2005-10-07,NaN,2005.0
1,673213.0,RCBP CAMPINAS-UNICAMP,FEMININO,1950-05-24,53.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,MORTO,NaN,2007-01-30,2007-01-30,2003-07-18,NaN,2003.0
2,673396.0,RCBP CAMPINAS-UNICAMP,FEMININO,1951-09-02,51.0,PARDA,BRASIL,S�O PAULO,NaN,NaN,...,NaN,NaN,NaN,MORTO,C�NCER,2005-07-03,2005-07-03,2003-05-16,NaN,2003.0
3,673425.0,RCBP CAMPINAS-UNICAMP,FEMININO,1929-11-21,73.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,VIVO,NaN,NaN,2010-07-28,2003-04-01,NaN,2003.0
4,673745.0,RCBP CAMPINAS-UNICAMP,MASCULINO,1934-07-03,71.0,BRANCO,BRASIL,S�O PAULO,NaN,NaN,...,NaN,NaN,NaN,MORTO,C�NCER,2005-10-21,2005-10-21,2005-10-21,NaN,2005.0


In [3]:
df.shape

(1778176, 38)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1778176 entries, 0 to 1778175
Data columns (total 38 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   Patient.Code                     float64
 1   RCBP.Name                        object 
 2   Gender                           object 
 3   Date.of.Birth                    object 
 4   Age                              float64
 5   Raca.Color                       object 
 6   Nationality                      object 
 7   Naturality.State                 object 
 8   Naturality                       object 
 9   Degree.of.Education              object 
 10  State.Civil                      object 
 11  Code.Profession                  float64
 12  Name.Occupation                  object 
 13  Status.Address                   object 
 14  City.Address                     object 
 15  Description.of.Topography        object 
 16  Topography.Code                  object 
 17  Morpholo

In [7]:
df['Status.Vital'].value_counts()

MORTO    123862
VIVO      85896
Name: Status.Vital, dtype: int64

In [8]:
df['Type.of.Death'].value_counts()

C�NCER        375903
N�O C�NCER     23376
Name: Type.of.Death, dtype: int64

In [11]:
df.Nationality.value_counts()

BRASIL           811183
PORTUGAL            749
ITALIA              451
JAPAO               448
ESPANHA             266
                  ...  
BAREIN                1
AFEGANISTAO           1
SUDAO                 1
ILHAS NORFOLK         1
ANTIGUA               1
Name: Nationality, Length: 113, dtype: int64

In [14]:
df.isna().sum()

Patient.Code                         74429
RCBP.Name                                0
Gender                                   0
Date.of.Birth                       768445
Age                                  79972
Raca.Color                          785048
Nationality                         962996
Naturality.State                   1326620
Naturality                         1591764
Degree.of.Education                1372348
State.Civil                        1179041
Code.Profession                     140150
Name.Occupation                     140150
Status.Address                       64197
City.Address                         63842
Description.of.Topography                0
Topography.Code                          0
Morphology.Description                   0
Code.of.Morphology                       0
Description.of.Disease                   0
Illness.Code                             0
Child.Illness.Description          1079470
Child.Illness.Code                 1079470
Youth.Adult

In [15]:
df['Type.of.Death']

0          C�NCER
1             NaN
2          C�NCER
3             NaN
4          C�NCER
            ...  
1778171       NaN
1778172       NaN
1778173       NaN
1778174       NaN
1778175       NaN
Name: Type.of.Death, Length: 1778176, dtype: object

In [19]:
df['Status.Vital'].isna().sum()

1568418

In [17]:
# Explain a lot of the data is missing, for cancer and death (thats why I have to drop about 80% of the data)

In [20]:
df_cancer = df.copy()

## I'll use this for just looking for cancer or no cancer 

In [39]:
df_cancer = df_cancer[df_cancer['Type.of.Death'].notna()]

In [21]:
df_cancer.shape

(1778176, 38)

In [22]:
df_cancer.head()

,Patient.Code,RCBP.Name,Gender,Date.of.Birth,Age,Raca.Color,Nationality,Naturality.State,Naturality,Degree.of.Education,...,Laterality,Statement,TNM,Status.Vital,Type.of.Death,Date.of.Death,Date.of.Last.Contact,Date.of.Diagnostic,Distant.metastasis,year
0,672054.0,RCBP CAMPINAS-UNICAMP,MASCULINO,1975-03-16,30.0,BRANCO,BRASIL,S�O PAULO,NaN,NaN,...,NaN,NaN,NaN,MORTO,C�NCER,2005-10-07,2005-10-07,2005-10-07,NaN,2005.0
1,673213.0,RCBP CAMPINAS-UNICAMP,FEMININO,1950-05-24,53.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,MORTO,NaN,2007-01-30,2007-01-30,2003-07-18,NaN,2003.0
2,673396.0,RCBP CAMPINAS-UNICAMP,FEMININO,1951-09-02,51.0,PARDA,BRASIL,S�O PAULO,NaN,NaN,...,NaN,NaN,NaN,MORTO,C�NCER,2005-07-03,2005-07-03,2003-05-16,NaN,2003.0
3,673425.0,RCBP CAMPINAS-UNICAMP,FEMININO,1929-11-21,73.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,VIVO,NaN,NaN,2010-07-28,2003-04-01,NaN,2003.0
4,673745.0,RCBP CAMPINAS-UNICAMP,MASCULINO,1934-07-03,71.0,BRANCO,BRASIL,S�O PAULO,NaN,NaN,...,NaN,NaN,NaN,MORTO,C�NCER,2005-10-21,2005-10-21,2005-10-21,NaN,2005.0


In [24]:
df_cancer['Type.of.Death'].value_counts()

C�NCER        375903
N�O C�NCER     23376
Name: Type.of.Death, dtype: int64

## Will use this df first for life / death

In [139]:
df = df[df['Status.Vital'].notna()]

In [140]:
df.shape

(209758, 38)

In [141]:
df = df.drop(columns=['Child.Illness.Description', 'Child.Illness.Code', 'Description.of.Disease',
                'Illness.Code', 'Youth.Adult.Illness.Description', 'Code.of.Disease.Adult.Young.',
                'Indicator.of.Rare.Case', 'Patient.Code' , 'RCBP.Name', 'Date.of.Birth',
                'TNM', 'Extension', 'Statement', 'Topography.Code', 'Morphology.Description',
                'Code.of.Morphology', 'Description.of.Disease', 'Illness.Code', 'Description.of.Topography',
                     'Date.of.Last.Contact', 'Date.of.Diagnostic', 'Distant.metastasis', 'Code.Profession',
                     'Name.Occupation','Laterality', 'Date.of.Death'])

In [142]:
df.head()

,Gender,Age,Raca.Color,Nationality,Naturality.State,Naturality,Degree.of.Education,State.Civil,Status.Address,City.Address,Diagnostic.means,Status.Vital,Type.of.Death,year
0,MASCULINO,30.0,BRANCO,BRASIL,S�O PAULO,NaN,NaN,CASADO,S�O PAULO,CAMPINAS,SDO,MORTO,C�NCER,2005.0
1,FEMININO,53.0,NaN,NaN,NaN,NaN,NaN,NaN,S�O PAULO,CAMPINAS,HISTOLOGIA DO TUMOR PRIM�RIO,MORTO,NaN,2003.0
2,FEMININO,51.0,PARDA,BRASIL,S�O PAULO,NaN,NaN,SOLTEIRO,S�O PAULO,CAMPINAS,HISTOLOGIA DO TUMOR PRIM�RIO,MORTO,C�NCER,2003.0
3,FEMININO,73.0,NaN,NaN,NaN,NaN,NaN,NaN,S�O PAULO,CAMPINAS,HISTOLOGIA DO TUMOR PRIM�RIO,VIVO,NaN,2003.0
4,MASCULINO,71.0,BRANCO,BRASIL,S�O PAULO,NaN,NaN,SOLTEIRO,S�O PAULO,CAMPINAS,SDO,MORTO,C�NCER,2005.0


In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209758 entries, 0 to 1759896
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Gender               209758 non-null  object 
 1   Age                  209443 non-null  float64
 2   Raca.Color           177827 non-null  object 
 3   Nationality          203393 non-null  object 
 4   Naturality.State     86722 non-null   object 
 5   Naturality           42211 non-null   object 
 6   Degree.of.Education  140833 non-null  object 
 7   State.Civil          153185 non-null  object 
 8   Status.Address       209757 non-null  object 
 9   City.Address         209758 non-null  object 
 10  Diagnostic.means     208974 non-null  object 
 11  Status.Vital         209758 non-null  object 
 12  Type.of.Death        120445 non-null  object 
 13  year                 209758 non-null  float64
dtypes: float64(2), object(12)
memory usage: 24.0+ MB


In [144]:
df.isna().sum()

Gender                      0
Age                       315
Raca.Color              31931
Nationality              6365
Naturality.State       123036
Naturality             167547
Degree.of.Education     68925
State.Civil             56573
Status.Address              1
City.Address                0
Diagnostic.means          784
Status.Vital                0
Type.of.Death           89313
year                        0
dtype: int64

In [145]:
df.Naturality.value_counts()

JA�                         4608
BARRETOS                    2797
CURITIBA                    2011
PO�OS DE CALDAS             1440
ARACAJU                     1333
                            ... 
ITAPEMA                        1
BAIAN�POLIS                    1
CAPIT�O LE�NIDAS MARQUES       1
S�O JOS� DO RIO CLARO          1
UBA�RA                         1
Name: Naturality, Length: 2723, dtype: int64

In [146]:
df['Naturality.State'].value_counts()

S�O PAULO              23085
PARAN�                 16606
MINAS GERAIS           12287
ALAGOAS                 5363
RIO GRANDE DO SUL       5355
SERGIPE                 3201
SANTA CATARINA          2930
PAR�                    2680
CEAR�                   2226
BAHIA                   2020
RIO DE JANEIRO          1558
AMAZONAS                1421
PERNAMBUCO              1379
ESPIRITO SANTO          1165
PARA�BA                  871
GOIAS                    757
MATO GROSSO              750
ACRE                     604
MARANH�O                 569
PIAU�                    525
MATO GROSSO DO SUL       445
DISTRITO FEDERAL         368
RIO GRANDE DO NORTE      361
TOCANTINS                 86
RONDONIA                  45
RORAIMA                   34
AMAPA                     31
Name: Naturality.State, dtype: int64

In [147]:
df['State.Civil'].value_counts()

CASADO                    75308
SOLTEIRO                  31012
VI�VO                     29771
SEPARADO JUDICIALMENTE    13290
UNI�O CONSENSUAL           3804
Name: State.Civil, dtype: int64

In [148]:
df['Type.of.Death'].value_counts()

C�NCER        112155
N�O C�NCER      8290
Name: Type.of.Death, dtype: int64

In [149]:
df['Raca.Color'].value_counts()

BRANCO      117315
PARDA        48908
PRETA         9232
AMARELA       2161
IND�GENA       211
Name: Raca.Color, dtype: int64

In [150]:
df['Age'].value_counts()

71.0     5211
69.0     5209
70.0     5206
68.0     5178
66.0     5130
         ... 
108.0       3
199.0       1
110.0       1
111.0       1
109.0       1
Name: Age, Length: 113, dtype: int64

In [151]:
df['Diagnostic.means'].value_counts()

HISTOLOGIA DO TUMOR PRIM�RIO    135684
SDO                              65686
PESQUISA                          3228
HISTOLOGIA DA MET�STASE           1737
CITOLOGIA                         1249
CL�NICO                           1213
MARCADORES TUMORAIS                177
Name: Diagnostic.means, dtype: int64

In [152]:
df['Degree.of.Education'].value_counts()

FUNDAMENTAL I (1� A 4� S�RIE)     49447
FUNDAMENTAL II (5� A 8� S�RIE)    27074
M�DIO (ANTIGO SEGUNDO GRAU)       26927
SUPERIOR COMPLETO                 19583
SEM ESCOLARIDADE                  15594
SUPERIOR INCOMPLETO                2208
Name: Degree.of.Education, dtype: int64

In [153]:
df = df[df['Age'] <= 120]

In [154]:
df = df.drop(columns = ['Naturality','Naturality.State'])

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209442 entries, 0 to 1759896
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Gender               209442 non-null  object 
 1   Age                  209442 non-null  float64
 2   Raca.Color           177753 non-null  object 
 3   Nationality          203084 non-null  object 
 4   Degree.of.Education  140812 non-null  object 
 5   State.Civil          153138 non-null  object 
 6   Status.Address       209441 non-null  object 
 7   City.Address         209442 non-null  object 
 8   Diagnostic.means     208658 non-null  object 
 9   Status.Vital         209442 non-null  object 
 10  Type.of.Death        120406 non-null  object 
 11  year                 209442 non-null  float64
dtypes: float64(2), object(10)
memory usage: 20.8+ MB


In [156]:
df.isna().sum()

Gender                     0
Age                        0
Raca.Color             31689
Nationality             6358
Degree.of.Education    68630
State.Civil            56304
Status.Address             1
City.Address               0
Diagnostic.means         784
Status.Vital               0
Type.of.Death          89036
year                       0
dtype: int64

In [157]:
df = df[df['Type.of.Death'].notna()]

In [158]:
df.isna().sum()

Gender                     0
Age                        0
Raca.Color              9302
Nationality             2282
Degree.of.Education    28132
State.Civil            17140
Status.Address             0
City.Address               0
Diagnostic.means         680
Status.Vital               0
Type.of.Death              0
year                       0
dtype: int64

In [159]:
df.shape

(120406, 12)

In [160]:
df['Degree.of.Education'].value_counts()

FUNDAMENTAL I (1� A 4� S�RIE)     30599
FUNDAMENTAL II (5� A 8� S�RIE)    18929
M�DIO (ANTIGO SEGUNDO GRAU)       17260
SEM ESCOLARIDADE                  12662
SUPERIOR COMPLETO                 12018
SUPERIOR INCOMPLETO                 806
Name: Degree.of.Education, dtype: int64

In [166]:
df['Degree.of.Education'] = df['Degree.of.Education'].replace(regex={r'^FUNDAMENTAL I .1. A 4. S.RIE.$': 'Fundamental I',
                  r'^FUNDAMENTAL II .5. A 8. S.RIE.$': 'Fundamental II',
                 r'^M.DIO .ANTIGO SEGUNDO GRAU.$': 'Former Second Degree',
                 'SEM ESCOLARIDADE': 'Without Education', 'SUPERIOR COMPLETO' : 'Graduated',
                 'SUPERIOR INCOMPLETO' : 'Incomplete High'})

In [167]:
df['Degree.of.Education'].value_counts()

Fundamental I           30599
Fundamental II          18929
Former Second Degree    17260
Without Education       12662
Graduated               12018
Incomplete High           806
Name: Degree.of.Education, dtype: int64

In [168]:
df.head()

,Gender,Age,Raca.Color,Nationality,Degree.of.Education,State.Civil,Status.Address,City.Address,Diagnostic.means,Status.Vital,Type.of.Death,year
0,MASCULINO,30.0,BRANCO,BRASIL,NaN,CASADO,S�O PAULO,CAMPINAS,SDO,MORTO,C�NCER,2005.0
2,FEMININO,51.0,PARDA,BRASIL,NaN,SOLTEIRO,S�O PAULO,CAMPINAS,HISTOLOGIA DO TUMOR PRIM�RIO,MORTO,C�NCER,2003.0
4,MASCULINO,71.0,BRANCO,BRASIL,NaN,SOLTEIRO,S�O PAULO,CAMPINAS,SDO,MORTO,C�NCER,2005.0
5,MASCULINO,71.0,BRANCO,BRASIL,Graduated,VI�VO,S�O PAULO,CAMPINAS,SDO,MORTO,C�NCER,2005.0
6,FEMININO,57.0,PARDA,BRASIL,NaN,CASADO,S�O PAULO,CAMPINAS,SDO,MORTO,C�NCER,2005.0


In [ ]:
|